In [ ]:
from skyfield.api import load

# Load planetary data and time
planets = load('de421.bsp')
ts = load.timescale()
t = ts.now()

# Get Earth and Mars
earth = planets['earth']
mars = planets['mars']

# Observe Mars from Earth
astrometric = earth.at(t).observe(mars)
ra, dec, distance = astrometric.radec()

# Print results
print(f"Mars Right Ascension: {ra}")
print(f"Mars Declination: {dec}")
print(f"Distance (AU): {distance}")


In [ ]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QLabel, QComboBox
)
from PyQt5.QtCore import QTimer
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D
from skyfield.api import load
import numpy as np
import matplotlib.pyplot as plt

planets = load('de421.bsp')
ts = load.timescale()

planet_names = {
    'Sun': 'sun',
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Earth': 'earth',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}

# Approximate planet radii in Earth radii (for marker size)
planet_radii = {
    'Sun': 109,
    'Mercury': 0.38,
    'Venus': 0.95,
    'Earth': 1.0,
    'Mars': 0.53,
    'Jupiter': 11.2,
    'Saturn': 9.45,
    'Uranus': 4.0,
    'Neptune': 3.88,
    'Pluto': 0.18
}

# Orbital periods in days for each planet (approximate)
orbital_periods_days = {
    'Mercury': 87.969,
    'Venus': 224.701,
    'Earth': 365.256,
    'Mars': 686.980,
    'Jupiter': 4332.59,
    'Saturn': 10759.22,
    'Uranus': 30688.5,
    'Neptune': 60182,
    'Pluto': 90560
}

class PlanetTracker3DFullOrbit(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Planet Tracker with Full Orbital Paths")
        self.setGeometry(100, 100, 1200, 900)

        layout = QVBoxLayout()

        self.dropdown = QComboBox()
        self.dropdown.addItems(['All Planets'] + list(planet_names.keys()))
        self.dropdown.currentIndexChanged.connect(self.update_data)

        self.ra_label = QLabel("Right Ascension: ")
        self.dec_label = QLabel("Declination: ")
        self.dist_label = QLabel("Distance (AU): ")

        self.figure = Figure(figsize=(12, 10))
        self.canvas = FigureCanvas(self.figure)
        self.ax = self.figure.add_subplot(111, projection='3d')

        layout.addWidget(self.dropdown)
        layout.addWidget(self.ra_label)
        layout.addWidget(self.dec_label)
        layout.addWidget(self.dist_label)
        layout.addWidget(self.canvas)
        self.setLayout(layout)

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_data)
        self.timer.start(1000)

        self.canvas.mpl_connect('scroll_event', self.on_scroll)

        self._zoom_scale = 1.0
        self._base_limit = 35

        self.update_data()

    def on_scroll(self, event):
        scale_factor = 0.9 if event.button == 'up' else 1.1
        self._zoom_scale *= scale_factor

        new_limit = self._base_limit * self._zoom_scale
        self.ax.set_xlim(-new_limit, new_limit)
        self.ax.set_ylim(-new_limit, new_limit)
        self.ax.set_zlim(-new_limit, new_limit)
        self.canvas.draw()

    def plot_full_orbit(self, target, name, color):
        sun = planets['sun']

        # Ephemeris valid time range
        ephem_start = ts.utc(1899, 7, 29)
        ephem_end = ts.utc(2053, 10, 9)

        if name == 'Sun':
            return  # no orbit for the Sun

        # Start date for orbit - keep inside ephemeris range
        start_time = ts.utc(2000, 1, 1)
        if start_time < ephem_start:
            start_time = ephem_start
        if start_time > ephem_end:
            start_time = ephem_end

        days = orbital_periods_days.get(name, 365.25)
        end_time_candidate = start_time + days

        # Clip to ephemeris range
        if end_time_candidate > ephem_end:
            end_time = ephem_end
        else:
            end_time = end_time_candidate

        # Number of days we can plot (might be less than full orbit)
        plot_days = end_time - start_time  # This is already a float number of days


        # If plot_days too small, fallback to 1 year
        if plot_days < 50:
            start_time = ephem_start
            end_time = ephem_start + 365.25
            plot_days = 365.25

        times = ts.linspace(start_time, end_time, 500)

        positions = sun.at(times).observe(target).position.au

        self.ax.plot(positions[0], positions[1], positions[2],
                    color=color, linewidth=0.8, alpha=0.6)

        # current position at now()
        t_now = ts.now()
        current_pos = sun.at(t_now).observe(target).position.au
        size = planet_radii.get(name, 1) * 5
        self.ax.scatter(*current_pos, s=size, color=color, label=name)
        self.ax.text(current_pos[0], current_pos[1], current_pos[2], name, fontsize=8)

        return positions


    def update_data(self):
        selected = self.dropdown.currentText()
        sun = planets['sun']

        self.ax.clear()
        self.ax.set_title("Solar System 3D View with Full Orbital Paths (AU)")
        self.ax.set_xlabel('X (AU)')
        self.ax.set_ylabel('Y (AU)')
        self.ax.set_zlabel('Z (AU)')
        self.ax.grid(True)

        sun_size = planet_radii['Sun'] * 5
        self.ax.scatter(0, 0, 0, color='yellow', s=sun_size,
                        label='Sun', edgecolors='orange', linewidths=1)
        self.ax.text(0.2, 0.2, 0.2, 'Sun', fontsize=9)

        if selected == 'All Planets':
            max_range = 0
            for idx, (name, sf_name) in enumerate(planet_names.items()):
                if sf_name == 'sun':
                    continue
                target = planets[sf_name]
                color = plt.cm.tab10(idx % 10)
                orbit_positions = self.plot_full_orbit(target, name, color)
                if orbit_positions is not None:
                    max_range = max(max_range, np.max(np.linalg.norm(orbit_positions, axis=0)))

            max_range = max(max_range, 30) * 1.1
            self._base_limit = max_range
            limit = self._base_limit * self._zoom_scale
            self.ax.set_xlim(-limit, limit)
            self.ax.set_ylim(-limit, limit)
            self.ax.set_zlim(-limit, limit)

            self.ra_label.setText("Right Ascension: N/A")
            self.dec_label.setText("Declination: N/A")
            self.dist_label.setText("Distance (AU): N/A")

        else:
            if selected == 'Sun':
                self.ra_label.setText("Right Ascension: N/A")
                self.dec_label.setText("Declination: N/A")
                self.dist_label.setText("Distance (AU): 0.0000")
                self.ax.set_xlim(-1, 1)
                self.ax.set_ylim(-1, 1)
                self.ax.set_zlim(-1, 1)
            else:
                target = planets[planet_names[selected]]
                astrometric = sun.at(ts.now()).observe(target)
                ra, dec, distance = astrometric.radec()
                pos = astrometric.position.au
                size = planet_radii.get(selected, 1) * 5
                color = 'red'
                self.ax.scatter(*pos, s=size, color=color, label=selected)
                self.ax.text(pos[0], pos[1], pos[2], selected, fontsize=10)

                self.plot_full_orbit(target, selected, color)

                max_range = max(abs(c) for c in pos) * 1.2
                max_range = max(max_range, 1)
                self._base_limit = max_range
                limit = self._base_limit * self._zoom_scale
                self.ax.set_xlim(-limit, limit)
                self.ax.set_ylim(-limit, limit)
                self.ax.set_zlim(-limit, limit)

                self.ra_label.setText(f"Right Ascension: {ra}")
                self.dec_label.setText(f"Declination: {dec}")
                self.dist_label.setText(f"Distance (AU): {distance.au:.4f}")

        self.ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        self.canvas.draw()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    tracker = PlanetTracker3DFullOrbit()
    tracker.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QLabel, QComboBox
)
from PyQt5.QtCore import QTimer
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D
from skyfield.api import load
import numpy as np
import matplotlib.pyplot as plt

planets = load('de421.bsp')
ts = load.timescale()

planet_names = {
    'Sun': 'sun',
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Earth': 'earth',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}

planet_radii = {
    'Sun': 109,
    'Mercury': 0.38,
    'Venus': 0.95,
    'Earth': 1.0,
    'Mars': 0.53,
    'Jupiter': 11.2,
    'Saturn': 9.45,
    'Uranus': 4.0,
    'Neptune': 3.88,
    'Pluto': 0.18
}

orbital_periods_days = {
    'Mercury': 87.969,
    'Venus': 224.701,
    'Earth': 365.256,
    'Mars': 686.980,
    'Jupiter': 4332.59,
    'Saturn': 10759.22,
    'Uranus': 30688.5,
    'Neptune': 60182,
    'Pluto': 90560
}

class PlanetTracker3DFullOrbit(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Planet Tracker with Full Orbital Paths")
        self.setGeometry(100, 100, 1200, 900)

        layout = QVBoxLayout()

        self.dropdown = QComboBox()
        self.dropdown.addItems(['All Planets'] + list(planet_names.keys()))
        self.dropdown.currentIndexChanged.connect(self.update_data)

        self.ra_label = QLabel("Right Ascension: ")
        self.dec_label = QLabel("Declination: ")
        self.dist_label = QLabel("Distance (AU): ")

        self.figure = Figure(figsize=(12, 10))
        self.canvas = FigureCanvas(self.figure)
        self.ax = self.figure.add_subplot(111, projection='3d')

        layout.addWidget(self.dropdown)
        layout.addWidget(self.ra_label)
        layout.addWidget(self.dec_label)
        layout.addWidget(self.dist_label)
        layout.addWidget(self.canvas)
        self.setLayout(layout)

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_data)
        self.timer.start(1000)

        self.canvas.mpl_connect('scroll_event', self.on_scroll)

        self._zoom_scale = 1.0
        self._base_limit = 35

        self.update_data()

    def on_scroll(self, event):
        scale_factor = 0.9 if event.button == 'up' else 1.1
        self._zoom_scale *= scale_factor

        new_limit = self._base_limit * self._zoom_scale
        self.ax.set_xlim(-new_limit, new_limit)
        self.ax.set_ylim(-new_limit, new_limit)
        self.ax.set_zlim(-new_limit, new_limit)
        self.canvas.draw()

    def plot_star_field(self, num_stars=1000, limit=100):
        np.random.seed(42)
        xs = np.random.uniform(-limit, limit, num_stars)
        ys = np.random.uniform(-limit, limit, num_stars)
        zs = np.random.uniform(-limit, limit, num_stars)
        self.ax.scatter(xs, ys, zs, color='white', s=0.5, alpha=0.4, zorder=0)

    def plot_full_orbit(self, target, name, color):
        sun = planets['sun']

        ephem_start = ts.utc(1899, 7, 29)
        ephem_end = ts.utc(2053, 10, 9)

        if name == 'Sun':
            return

        start_time = ts.utc(2000, 1, 1)
        if start_time < ephem_start:
            start_time = ephem_start
        if start_time > ephem_end:
            start_time = ephem_end

        days = orbital_periods_days.get(name, 365.25)
        end_time_candidate = start_time + days

        if end_time_candidate > ephem_end:
            end_time = ephem_end
        else:
            end_time = end_time_candidate

        plot_days = end_time - start_time
        if plot_days < 50:
            start_time = ephem_start
            end_time = ephem_start + 365.25
            plot_days = 365.25

        times = ts.linspace(start_time, end_time, 500)
        positions = sun.at(times).observe(target).position.au

        self.ax.plot(positions[0], positions[1], positions[2],
                     color=color, linewidth=0.8, alpha=0.6)

        t_now = ts.now()
        current_pos = sun.at(t_now).observe(target).position.au
        size = planet_radii.get(name, 1) * 5
        self.ax.scatter(*current_pos, s=size, color=color, label=name)
        self.ax.text(current_pos[0], current_pos[1], current_pos[2], name, fontsize=8)

        return positions

    def update_data(self):
        selected = self.dropdown.currentText()
        sun = planets['sun']

        self.ax.clear()
        self.figure.patch.set_facecolor('black')
        self.ax.set_facecolor('black')
        self.plot_star_field(num_stars=2000, limit=200)

        self.ax.set_title("Solar System 3D View with Full Orbital Paths (AU)", color='white')
        self.ax.set_xlabel('X (AU)', color='white')
        self.ax.set_ylabel('Y (AU)', color='white')
        self.ax.set_zlabel('Z (AU)', color='white')
        self.ax.grid(True, color='gray', linestyle='dotted', alpha=0.3)

        self.ax.tick_params(colors='white')

        sun_size = planet_radii['Sun'] * 5
        self.ax.scatter(0, 0, 0, color='yellow', s=sun_size,
                        label='Sun', edgecolors='orange', linewidths=1)
        self.ax.text(0.2, 0.2, 0.2, 'Sun', fontsize=9, color='white')

        if selected == 'All Planets':
            max_range = 0
            for idx, (name, sf_name) in enumerate(planet_names.items()):
                if sf_name == 'sun':
                    continue
                target = planets[sf_name]
                color = plt.cm.tab10(idx % 10)
                orbit_positions = self.plot_full_orbit(target, name, color)
                if orbit_positions is not None:
                    max_range = max(max_range, np.max(np.linalg.norm(orbit_positions, axis=0)))

            max_range = max(max_range, 30) * 1.1
            self._base_limit = max_range
            limit = self._base_limit * self._zoom_scale
            self.ax.set_xlim(-limit, limit)
            self.ax.set_ylim(-limit, limit)
            self.ax.set_zlim(-limit, limit)

            self.ra_label.setText("Right Ascension: N/A")
            self.dec_label.setText("Declination: N/A")
            self.dist_label.setText("Distance (AU): N/A")

        else:
            if selected == 'Sun':
                self.ra_label.setText("Right Ascension: N/A")
                self.dec_label.setText("Declination: N/A")
                self.dist_label.setText("Distance (AU): 0.0000")
                self.ax.set_xlim(-1, 1)
                self.ax.set_ylim(-1, 1)
                self.ax.set_zlim(-1, 1)
            else:
                target = planets[planet_names[selected]]
                astrometric = sun.at(ts.now()).observe(target)
                ra, dec, distance = astrometric.radec()
                pos = astrometric.position.au
                size = planet_radii.get(selected, 1) * 5
                color = 'red'
                self.ax.scatter(*pos, s=size, color=color, label=selected)
                self.ax.text(pos[0], pos[1], pos[2], selected, fontsize=10, color='white')

                self.plot_full_orbit(target, selected, color)

                max_range = max(abs(c) for c in pos) * 1.2
                max_range = max(max_range, 1)
                self._base_limit = max_range
                limit = self._base_limit * self._zoom_scale
                self.ax.set_xlim(-limit, limit)
                self.ax.set_ylim(-limit, limit)
                self.ax.set_zlim(-limit, limit)

                self.ra_label.setText(f"Right Ascension: {ra}")
                self.dec_label.setText(f"Declination: {dec}")
                self.dist_label.setText(f"Distance (AU): {distance.au:.4f}")

        self.ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), labelcolor='white')
        self.canvas.draw()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    tracker = PlanetTracker3DFullOrbit()
    tracker.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QLabel, QComboBox
)
from PyQt5.QtCore import QTimer
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D
from skyfield.api import load
import numpy as np
import matplotlib.pyplot as plt

planets = load('de421.bsp')
ts = load.timescale()

planet_names = {
    'Sun': 'sun',
    'Mercury': 'mercury',
    'Venus': 'venus',
    'Earth': 'earth',
    'Mars': 'mars',
    'Jupiter': 'jupiter barycenter',
    'Saturn': 'saturn barycenter',
    'Uranus': 'uranus barycenter',
    'Neptune': 'neptune barycenter',
    'Pluto': 'pluto barycenter'
}

planet_radii = {
    'Sun': 109,
    'Mercury': 0.38,
    'Venus': 0.95,
    'Earth': 1.0,
    'Mars': 0.53,
    'Jupiter': 11.2,
    'Saturn': 9.45,
    'Uranus': 4.0,
    'Neptune': 3.88,
    'Pluto': 0.18
}

orbital_periods_days = {
    'Mercury': 87.969,
    'Venus': 224.701,
    'Earth': 365.256,
    'Mars': 686.980,
    'Jupiter': 4332.59,
    'Saturn': 10759.22,
    'Uranus': 30688.5,
    'Neptune': 60182,
    'Pluto': 90560
}

class PlanetTracker3DFullOrbit(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Planet Tracker with Full Orbital Paths")
        self.setGeometry(100, 100, 1200, 900)

        layout = QVBoxLayout()

        self.dropdown = QComboBox()
        self.dropdown.addItems(['All Planets'] + list(planet_names.keys()))
        self.dropdown.currentIndexChanged.connect(self.update_data)

        self.ra_label = QLabel("Right Ascension: ")
        self.dec_label = QLabel("Declination: ")
        self.dist_label = QLabel("Distance (AU): ")

        self.figure = Figure(figsize=(12, 10))
        self.canvas = FigureCanvas(self.figure)
        self.ax = self.figure.add_subplot(111, projection='3d')

        layout.addWidget(self.dropdown)
        layout.addWidget(self.ra_label)
        layout.addWidget(self.dec_label)
        layout.addWidget(self.dist_label)
        layout.addWidget(self.canvas)
        self.setLayout(layout)

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_data)
        self.timer.start(1000)

        self.canvas.mpl_connect('scroll_event', self.on_scroll)

        self._zoom_scale = 1.0
        self._base_limit = 35

        self.update_data()

    def on_scroll(self, event):
        scale_factor = 0.9 if event.button == 'up' else 1.1
        self._zoom_scale *= scale_factor

        new_limit = self._base_limit * self._zoom_scale
        self.ax.set_xlim(-new_limit, new_limit)
        self.ax.set_ylim(-new_limit, new_limit)
        self.ax.set_zlim(-new_limit, new_limit)
        self.canvas.draw()

    def plot_star_field(self, num_stars=1000, limit=100):
        np.random.seed(42)
        xs = np.random.uniform(-limit, limit, num_stars)
        ys = np.random.uniform(-limit, limit, num_stars)
        zs = np.random.uniform(-limit, limit, num_stars)
        self.ax.scatter(xs, ys, zs, color='white', s=0.5, alpha=0.4, zorder=0)

    def plot_full_orbit(self, target, name, color):
        sun = planets['sun']

        ephem_start = ts.utc(1899, 7, 29)
        ephem_end = ts.utc(2053, 10, 9)

        if name == 'Sun':
            return

        start_time = ts.utc(2000, 1, 1)
        if start_time < ephem_start:
            start_time = ephem_start
        if start_time > ephem_end:
            start_time = ephem_end

        days = orbital_periods_days.get(name, 365.25)
        end_time_candidate = start_time + days

        if end_time_candidate > ephem_end:
            end_time = ephem_end
        else:
            end_time = end_time_candidate

        plot_days = end_time - start_time
        if plot_days < 50:
            start_time = ephem_start
            end_time = ephem_start + 365.25
            plot_days = 365.25

        times = ts.linspace(start_time, end_time, 500)
        positions = sun.at(times).observe(target).position.au

        self.ax.plot(positions[0], positions[1], positions[2],
                     color=color, linewidth=0.8, alpha=0.6)

        t_now = ts.now()
        current_pos = sun.at(t_now).observe(target).position.au
        size = planet_radii.get(name, 1) * 5
        self.ax.scatter(*current_pos, s=size, color=color, label=name)
        self.ax.text(current_pos[0], current_pos[1], current_pos[2], name, fontsize=8, color='white')

        return positions

    def update_data(self):
        selected = self.dropdown.currentText()
        sun = planets['sun']

        self.ax.clear()
        self.figure.patch.set_facecolor('black')
        self.ax.set_facecolor('black')

        # Hide axes, grid, ticks, labels — keep only celestial objects visible
        self.ax.set_axis_off()
        self.ax.grid(False)

        self.plot_star_field(num_stars=2000, limit=200)

        sun_size = planet_radii['Sun'] * 5
        self.ax.scatter(0, 0, 0, color='yellow', s=sun_size,
                        label='Sun', edgecolors='orange', linewidths=1)
        self.ax.text(0.2, 0.2, 0.2, 'Sun', fontsize=9, color='white')

        if selected == 'All Planets':
            max_range = 0
            for idx, (name, sf_name) in enumerate(planet_names.items()):
                if sf_name == 'sun':
                    continue
                target = planets[sf_name]
                color = plt.cm.tab10(idx % 10)
                orbit_positions = self.plot_full_orbit(target, name, color)
                if orbit_positions is not None:
                    max_range = max(max_range, np.max(np.linalg.norm(orbit_positions, axis=0)))

            max_range = max(max_range, 30) * 1.1
            self._base_limit = max_range
            limit = self._base_limit * self._zoom_scale
            self.ax.set_xlim(-limit, limit)
            self.ax.set_ylim(-limit, limit)
            self.ax.set_zlim(-limit, limit)

            self.ra_label.setText("Right Ascension: N/A")
            self.dec_label.setText("Declination: N/A")
            self.dist_label.setText("Distance (AU): N/A")

        else:
            if selected == 'Sun':
                self.ra_label.setText("Right Ascension: N/A")
                self.dec_label.setText("Declination: N/A")
                self.dist_label.setText("Distance (AU): 0.0000")
                self.ax.set_xlim(-1, 1)
                self.ax.set_ylim(-1, 1)
                self.ax.set_zlim(-1, 1)
            else:
                target = planets[planet_names[selected]]
                astrometric = sun.at(ts.now()).observe(target)
                ra, dec, distance = astrometric.radec()
                pos = astrometric.position.au
                size = planet_radii.get(selected, 1) * 5
                color = 'red'
                self.ax.scatter(*pos, s=size, color=color, label=selected)
                self.ax.text(pos[0], pos[1], pos[2], selected, fontsize=10, color='white')

                self.plot_full_orbit(target, selected, color)

                max_range = max(abs(c) for c in pos) * 1.2
                max_range = max(max_range, 1)
                self._base_limit = max_range
                limit = self._base_limit * self._zoom_scale
                self.ax.set_xlim(-limit, limit)
                self.ax.set_ylim(-limit, limit)
                self.ax.set_zlim(-limit, limit)

                self.ra_label.setText(f"Right Ascension: {ra}")
                self.dec_label.setText(f"Declination: {dec}")
                self.dist_label.setText(f"Distance (AU): {distance.au:.4f}")

        self.ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), labelcolor='white')
        self.canvas.draw()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    tracker = PlanetTracker3DFullOrbit()
    tracker.show()
    sys.exit(app.exec_())


In [1]:
import sys
import numpy as np
from PyQt5.QtWidgets import QApplication, QMainWindow
from PyQt5.QtCore import QTimer
import pyqtgraph as pg
import pyqtgraph.opengl as gl
from skyfield.api import load

class PlanetTrackerGL(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Planet Tracker with PyQtGraph")
        self.resize(1000, 800)

        self.view = gl.GLViewWidget()
        self.view.setCameraPosition(distance=50)
        self.setCentralWidget(self.view)

        self.planets = load('de421.bsp')
        self.ts = load.timescale()
        self.planet_names = ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn']
        self.planet_targets = [self.planets[name.lower()] for name in self.planet_names]
        self.planet_colors = [(1, 0.6, 0.0), (1, 1, 0), (0, 0.5, 1), (1, 0.3, 0.3), (0.7, 0.3, 1), (0.9, 0.9, 0.5)]
        self.planet_spheres = []

        # Starfield background
        stars = gl.GLScatterPlotItem(pos=np.random.uniform(-200, 200, size=(3000, 3)), size=0.4, color=(1, 1, 1, 0.3))
        self.view.addItem(stars)

        # Sun
        self.sun = gl.GLScatterPlotItem(pos=np.array([[0, 0, 0]]), size=6, color=(1, 1, 0, 1))
        self.view.addItem(self.sun)

        for color in self.planet_colors:
            sphere = gl.GLScatterPlotItem(pos=np.array([[0, 0, 0]]), size=2.5, color=color)
            self.view.addItem(sphere)
            self.planet_spheres.append(sphere)

        self.t = self.ts.now()
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_positions)
        self.timer.start(30)

    def update_positions(self):
        sun = self.planets['sun']
        self.t = self.ts.tt_jd(self.t.tt + 0.02)

        for i, target in enumerate(self.planet_targets):
            pos = sun.at(self.t).observe(target).position.au
            self.planet_spheres[i].setData(pos=np.array([pos]))

if __name__ == "__main__":
    app = QApplication(sys.argv)
    tracker = PlanetTrackerGL()
    tracker.show()
    sys.exit(app.exec_())


ModuleNotFoundError: No module named 'pyqtgraph'

In [ ]:
import sys
import random
import math
from PyQt5.QtCore import Qt, QTimer, QPointF
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QColor, QPen

class Particle:
    def __init__(self, x, y):
        self.pos = QPointF(x, y)
        self.vel = QPointF(random.uniform(-1, 1), random.uniform(-1, 1))

    def update(self, width, height):
        self.pos += self.vel
        if self.pos.x() < 0 or self.pos.x() > width:
            self.vel.setX(-self.vel.x())
        if self.pos.y() < 0 or self.pos.y() > height:
            self.vel.setY(-self.vel.y())

class ParticleNetwork(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Animated Particle Network")
        self.resize(800, 600)
        self.particles = [Particle(random.randint(0, 800), random.randint(0, 600)) for _ in range(100)]
        self.timer = QTimer()
        self.timer.timeout.connect(self.update)
        self.timer.start(16)

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.fillRect(self.rect(), Qt.black)

        for p in self.particles:
            p.update(self.width(), self.height())
            painter.setPen(Qt.NoPen)
            painter.setBrush(QColor(255, 255, 255, 180))
            painter.drawEllipse(p.pos, 2, 2)

        painter.setPen(QPen(QColor(255, 255, 255, 50), 1))
        for i, p1 in enumerate(self.particles):
            for j in range(i + 1, len(self.particles)):
                p2 = self.particles[j]
                dist = math.hypot(p1.pos.x() - p2.pos.x(), p1.pos.y() - p2.pos.y())
                if dist < 100:
                    alpha = max(0, 150 - dist)
                    painter.setPen(QPen(QColor(255, 255, 255, int(alpha)), 1))
                    painter.drawLine(p1.pos, p2.pos)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ParticleNetwork()
    window.show()
    sys.exit(app.exec_())


In [ ]:
import sys
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout
from PyQt5.QtCore import QTimer
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D
from skyfield.api import load
import matplotlib.pyplot as plt

class SmoothPlanetTracker(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Smooth Animated 3D Planet Tracker")
        self.setGeometry(100, 100, 1200, 900)

        layout = QVBoxLayout()
        self.canvas = FigureCanvas(Figure())
        layout.addWidget(self.canvas)
        self.setLayout(layout)

        self.ax = self.canvas.figure.add_subplot(111, projection='3d')
        self.ax.set_facecolor('black')
        self.canvas.figure.patch.set_facecolor('black')

        self.planets = load('de421.bsp')
        self.ts = load.timescale()
        self.planet_names = ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn']
        self.planet_targets = [self.planets[name.lower()] for name in self.planet_names]
        self.planet_colors = plt.cm.rainbow(np.linspace(0, 1, len(self.planet_names)))

        self.t_now = self.ts.now()
        self.timer = QTimer()
        self.timer.timeout.connect(self.animate)
        self.timer.start(30)

    def animate(self):
        self.ax.clear()
        self.ax.set_xlim(-35, 35)
        self.ax.set_ylim(-35, 35)
        self.ax.set_zlim(-35, 35)
        self.ax.set_axis_off()

        sun = self.planets['sun']
        self.ax.scatter(0, 0, 0, color='yellow', s=200, label='Sun')

        for i, target in enumerate(self.planet_targets):
            pos = sun.at(self.t_now).observe(target).position.au
            self.ax.scatter(*pos, color=self.planet_colors[i], s=50)
            self.ax.text(pos[0], pos[1], pos[2], self.planet_names[i], color='white', fontsize=8)

        self.t_now = self.ts.tt_jd(self.t_now.tt + 0.02)  # Smooth time step forward
        self.canvas.draw()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = SmoothPlanetTracker()
    window.show()
    sys.exit(app.exec_())
